# SecureNN

This notebook is just for exploration of the paper

In [1]:
bits = 64
m = 2**bits
p1 = m-1

print(f"Paper works in modulo 2**64 and 2**64-1")
print(f"2**64:\n\tdec={m}\n\tbin={bin(m)}")
print(f"2**64-1:\n\tdec={p1}\n\tbin={bin(p1)}")

Paper works in modulo 2**64 and 2**64-1
2**64:
	dec=18446744073709551616
	bin=0b10000000000000000000000000000000000000000000000000000000000000000
2**64-1:
	dec=18446744073709551615
	bin=0b1111111111111111111111111111111111111111111111111111111111111111


Let's now write a function that converts an integer into a 64 bit variable. This may be a little bit tricky because python does not allow to add padding zeroes by default when converting to binary and we want to have an exact representation of the integer in 64 bit.

In [2]:
def to_bin64(x: int) -> str:
    # Return x into 64 bit (8 byte)
    assert x<2**64, f"{x} has to be smaller than {2**64} and therefore cannot be represented as int64"
    
    b = bin(x)
    l = len(b)
    if l==66:
        return b
    else:
        n = 66-l
        return '0b' + '0'*n + b.split("b")[1]

The least significant bit is the last bit, that determines if the number is odd, on the contrary the most significant bit is the first representation bit, in our case is the bit (in our case the $2^{63}$ power)

In [3]:
def LSB(x: int) -> bool:
    # computes least significant digit of an integer
    # the first bit
    return x%2


def MSB(x: int) -> bool:
    assert x>0, "x has to be positive integer"
    b = to_bin64(x)
    assert len(b)==66
    
    return int(b[2])

According to the paper, if we are working on an odd ring $p_1=2^{64}-1$ then, MSB(a)=LSB(2a). I won't give a mathematical proof but here you can find a quick test running 500 examples of random values between 0 and $p_1$

In [4]:
from random import randrange

for i in range(500):
    a = randrange(0, p1)
    a2 = 2*a%p1
    
    msb = MSB(a)
    lsb = LSB(a2)

    assert msb==lsb, "MSB(a) != LSB(2*a), something went wrong"

An important ingredient of the protocol is the function wrap. Basically is a boolean telling us if the sum of two values exeeds the value of the field and therefore "wraps around".

In [5]:
def add_binary(x, y):
    # binary addition modulo 2**(len(x))
    # x and y are binary numbers
    
    x_ = x.split("b")[1]
    y_ = y.split("b")[1]
    
    len_x = len(x_)
    len_y = len(y_)
    
    # both numbers have to be the same ammount of bits
    assert len_x == len_y, f"x is length {len_x} and y is {len_y}, must be the same"
    
    result = ''
    carry = 0
    
    for i in range(len_x -1, -1, -1):
        r = carry
        r += 1 if x_[i] == '1' else 0
        r += 1 if y_[i] == '1' else 0
        result = ('1' if r % 2 == 1 else '0') + result 
        carry = 0 if r < 2 else 1

    # If we carry 1 we have to add 1 at the beginning of the string
    # this is, we are wrapping around the size
    #if carry != 0:
    #    return '0b'+ '1'+result
    
    # we don't want wrap arround but modulo 2**(len(x)) so we don't add 
    # the padding 1 in any case
    return '0b'+result

In [6]:
for _ in range(50):
    x = randrange(m)
    y = randrange(m)
    
    res = add_binary(to_bin64(x), to_bin64(y))
    assert int(res, 2)==(x+y)%m, f"something went wrong on the binary sum of {x} and {y}"

In [7]:
def wrap(x, y, L):
    return x+y>L-1

In [8]:
def share(x, L, parties=2):
    shares = [randrange(L) for _ in range(parties-1)]
    shares.append(x - sum(shares)%L)
    return shares


def reconstruct(shares, L, parties=2):
    return sum(shares)%L

s = share(5, 2**64)
reconstruct(s, 2**64)

5

In [9]:
wrap(3, 5, 151)

False

We have to write shares of binary numbers modulo a certain field, in the paper is 67.

* Convert integer to binary
* Share each bit over the field 67 (btw, this is prime number so it's really a field and not a ring anymore)


In [10]:
from random import randrange

def share_binary(x: int, field: int = 67):
    x_bin = to_bin64(x)
    share_0, share_1 = [], []
    for bit in x_bin[2:]:
        s0 = randrange(field)
        s1 = (int(bit)-s0)%field
        
        share_0.append(s0)
        share_1.append(s1)
        
    return share_0, share_1

In [11]:
x = randrange(2**64-1)
print(to_bin64(x))
share_0, share_1 = share_binary(x)

reconstruct = []
for e0, e1 in zip(share_0, share_1):
    reconstruct.append((e0+e1)%67)

print('0b'+''.join(str(x) for x in reconstruct))

0b1111111001110011110000011101110010110001010100101100101010100110
0b1111111001110011110000011101110010110001010100101100101010100110


In [12]:
s = to_bin64(randrange(67))
print(s)

0b0000000000000000000000000000000000000000000000000000000000000101
